In [3]:
from bs4 import BeautifulSoup
import time
from selenium import webdriver
import pandas as pd
from pathlib import Path
from IPython.display import display
from sqlalchemy import create_engine
import datetime

url = 'https://weather.com/id-ID/weather/tenday/l/Kramat+Daerah+Khusus+Ibukota+Jakarta?canonicalCityId=9d5f83a2a29b1f4aa5272fca53aa871b328c5ee66f6882d07480f0a94b37565c#detailIndex5'
url2 = 'https://www.tideschart.com/Indonesia/Jakarta/Weekly/'

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--disable-extensions")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--no-sandbox")
user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36'
chrome_options.add_argument(f'user-agent={user_agent}')

driver = webdriver.Chrome(chrome_options = chrome_options)
driver2 = webdriver.Chrome(chrome_options = chrome_options)

time.sleep(0.5)
driver.get(url)
driver2.get(url2)
time.sleep(0.5)

soup = BeautifulSoup(driver.page_source, 'lxml')
soup2 = BeautifulSoup(driver2.page_source, 'lxml')

df_01 = list()
for el in soup.find().find_all("summary",{"class" : "Disclosure--Summary--3GiL4 DaypartDetails--Summary--3Fuya Disclosure--hideBorderOnSummaryOpen--3_ZkO"}):
    df_01.append(el.get_text("\n",strip=True).split("\n"))

df_01 = pd.DataFrame(df_01)
df_01 = df_01.drop(df_01.columns[[2, 4, 6, 8, 9,11]], axis=1)
df_01 = df_01.drop(labels = [6,7,8,9,10,11,12,13], axis=0)
#display(df_01)

df_all = []

col_all =[
    "Datetime",
    "Hari",
    "Temperature_siang_Celcius",
    "Temperature_malam_Celcius",
    "Prediksi_cuaca",
    "Kemungkinan_hujan_persen",
    "Kecepatan_angin_km_per_hour",
    "Waktu_tide_1",
    "Tide_1_meter",
    "Waktu_tide_2",
    "Tide_2_meter",
    "Prediksi_cuaca_num"
]
df_all = pd.DataFrame(df_all, columns=col_all)

#display(df_all)

df_02 = list()

for el in soup2.find().find_all("table",{"class" : "table table-hover tidechart mb-4"}):
    fic2 = el.get_text("\n",strip=True)
    #print(fic)
    df_02.append(el.get_text("\n",strip=True).split("\n"))

df_02 = pd.DataFrame(df_02)

df_02_temp = []
df_02_temp = pd.DataFrame(df_02_temp, columns = ["0","1","2","3","4"])
df_02_temp = pd.DataFrame(df_02_temp)

a = 7
b = 8
c = 10
d = 11
e = 13

x = 1
y = 8
while x < y:
    val_col_02 = [
        df_02.iloc[0,a],
        df_02.iloc[0,b],
        df_02.iloc[0,c],
        df_02.iloc[0,d],
        df_02.iloc[0,e],
    ]
    df_02_temp.loc[len(df_02_temp)] = val_col_02
    x += 1
    a += 11
    b += 11
    c += 11
    d += 11
    e += 11
    
df_02 = df_02_temp.drop(labels = 0, axis=0)
df_02.index = range(len(df_02))
#df_02_temp.rename(df_02_temp.iloc[0], axis=0, inplace=True)
#df_02 = df_02_temp.drop(df_02_temp.index[0], axis = 0, inplace=True)
#display(df_02)

df_all["Hari"] = df_01.iloc[:,0]
df_all["Temperature_siang_Celcius"] = df_01.iloc[:,1]
df_all["Temperature_malam_Celcius"] = df_01.iloc[:,2]
df_all["Prediksi_cuaca"] = df_01.iloc[:,3]
df_all["Kemungkinan_hujan_persen"] = df_01.iloc[:,4]
df_all["Kecepatan_angin_km_per_hour"] = df_01.iloc[:,5]

df_all["Waktu_tide_1"] = df_02.iloc[:,1]
df_all["Tide_1_meter"] = df_02.iloc[:,2]
df_all["Waktu_tide_2"] =df_02.iloc[:,3]
df_all["Tide_2_meter"] = df_02.iloc[:,4]
#display(df_all)


df_all['Temperature_siang_Celcius'] = df_all.Temperature_siang_Celcius.str.findall('([-+]?\d*\.?\d+)')
df_all['Temperature_malam_Celcius'] = df_all.Temperature_malam_Celcius.str.findall('([-+]?\d*\.?\d+)')
df_all['Kemungkinan_hujan_persen'] = df_all.Kemungkinan_hujan_persen.str.findall('([-+]?\d*\.?\d+)')
df_all['Kecepatan_angin_km_per_hour'] = df_all.Kecepatan_angin_km_per_hour.str.findall('([-+]?\d*\.?\d+)')
df_all['Tide_1_meter'] = df_all.Tide_1_meter.str.findall('([-+]?\d*\.?\d+)')
df_all['Tide_2_meter'] = df_all.Tide_2_meter.str.findall('([-+]?\d*\.?\d+)')

#df_all.apply(pd.Series.explode)
#df_all.explode()
#display(df_all)
df_all = df_all.apply(pd.Series.explode)
#display(df_all)
df_all = df_all.drop([df_all.index[1], df_all.index[2], df_all.index[3], df_all.index[4], df_all.index[5]])
#df_all['id'] = 1
df_all.Temperature_siang_Celcius = pd.to_numeric(df_all.Temperature_siang_Celcius, errors='coerce')
df_all.Temperature_malam_Celcius = pd.to_numeric(df_all.Temperature_malam_Celcius, errors='coerce')
df_all.Kemungkinan_hujan_persen = pd.to_numeric(df_all.Kemungkinan_hujan_persen, errors='coerce')
df_all.Kecepatan_angin_km_per_hour = pd.to_numeric(df_all.Kecepatan_angin_km_per_hour, errors='coerce')
df_all.Tide_1_meter = pd.to_numeric(df_all.Tide_1_meter, errors='coerce')
df_all.Tide_2_meter = pd.to_numeric(df_all.Tide_2_meter, errors='coerce')
df_all['Datetime'] = datetime.datetime.now()
#df_all['Prediksi_cuaca_num'].mask((df_all['Prediksi_cuaca']) = 0.5, 0, inplace=True)
df_all.loc[df_all['Prediksi_cuaca'] == "Badai Petir Petang", 'Prediksi_cuaca_num'] = 1
df_all.loc[df_all['Prediksi_cuaca'] == "Sebagian Berawan", 'Prediksi_cuaca_num'] = 2
df_all.loc[df_all['Prediksi_cuaca'] == "Badai Petir", 'Prediksi_cuaca_num'] = 3
df_all.loc[df_all['Prediksi_cuaca'] == "Badai Petir Terpencar", 'Prediksi_cuaca_num'] = 4
display(df_all)

my_file = Path("data_weather_tides_harian.csv")
if my_file.is_file():
    df_csv_01 = pd.read_csv("data_weather_tides_harian.csv")
    #df_csv_01 = pd.concat([df_csv_01, df_all])
    result = pd.concat([df_csv_01, df_all]).drop_duplicates(subset=["Hari"], keep="last")
    #df_csv_01.merge(df_all, how='outer', on='Hari')
    result.to_csv("data_weather_tides_harian.csv", index = False)
else :
    df_all.to_csv("data_weather_tides_harian.csv", index = False)

from sqlalchemy.orm import sessionmaker
# Credentials to database connection
hostname="localhost"
dbname="workshop"
uname="root"
pwd="sanjii123"


# Create SQLAlchemy engine to connect to MySQL Database
engine = create_engine("mysql+pymysql://{user}:{pw}@{host}/{db}"
				.format(host=hostname, db=dbname, user=uname, pw=pwd))

session = sessionmaker(bind=engine)()

# Convert dataframe to sql table                                   
df_all.to_sql('parsing_cuaca_gelombang_current', engine, if_exists='replace', index=False)
df_all.to_sql('parsing_cuaca_gelombang_hystorian', engine, if_exists='append', index=False)


engine.dispose()
session.close()
driver.quit()

C:\Users\P5CD1\AppData\Local\Temp\ipykernel_3912\1585197377.py:21: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(chrome_options = chrome_options)
C:\Users\P5CD1\AppData\Local\Temp\ipykernel_3912\1585197377.py:22: DeprecationWarning: use options instead of chrome_options
  driver2 = webdriver.Chrome(chrome_options = chrome_options)


,Datetime,Hari,Temperature_siang_Celcius,Temperature_malam_Celcius,Prediksi_cuaca,Kemungkinan_hujan_persen,Kecepatan_angin_km_per_hour,Waktu_tide_1,Tide_1_meter,Waktu_tide_2,Tide_2_meter,Prediksi_cuaca_num
0,2023-01-10 11:41:50.263237,Rab 11,32,24,Sebagian Berawan,24,15,10:54am,1,10:15pm,0.3,2


In [38]:
from bs4 import BeautifulSoup
import time
from selenium import webdriver
import pandas as pd
from pathlib import Path
from IPython.display import display
from sqlalchemy import create_engine
import datetime

url = 'https://www.tideschart.com/Indonesia/Jakarta/Weekly/'

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--disable-extensions")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--no-sandbox")
user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36'
chrome_options.add_argument(f'user-agent={user_agent}')

driver = webdriver.Chrome(chrome_options = chrome_options)

time.sleep(0.5)
driver.get(url)
time.sleep(0.5)

soup = BeautifulSoup(driver.page_source, 'lxml')


df_01 = list()
for el in soup.find("div",{"class" : "card-box card-bottom pb-0"}).find_all("script"):
    #fic = text.get_text(",",strip=True)
    #print(fic)
    df_01.append(el.get_text(",{",strip=True).split(","))

df_01 = pd.DataFrame(df_01)
display(df_01)
#df_02 = pd.DataFrame(df_01.list())
#meta = driver.execute_script('tide')


#display(df_02)



C:\Users\P5CD1\AppData\Local\Temp\ipykernel_3912\2152820641.py:20: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(chrome_options = chrome_options)


,0,1,2,3,4,5,6,7,8,9,...,66,67,68,69,70,71,72,73,74,75
0,"const tideData = (data1 = {title: ""Daerah Khus...","unit: ""m""","\n currentDate: ""2023-01-10""","\n currentTime: ""12:55""","\n tide: [{date: ""2023-01-09""","time: ""23:36""",value: 0.3 },"{ date: ""2023-01-10""","time: ""10:49""",value: 1 },...,"sunset: ""18:15"" }","{ date: ""2023-01-15""","sunrise: ""05:49""","sunset: ""18:15"" }","{ date: ""2023-01-16""","sunrise: ""05:50""","sunset: ""18:15"" }]",\n });\n\n new VisTideChart({\n ...,\n data: tideData,\n });
1,function showHideLocations() {\n\n\t\t\t\t\t\t...,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [37]:
df_01.to_csv('tes.csv')

In [56]:
from bs4 import BeautifulSoup
import time
from selenium import webdriver
import pandas as pd
from pathlib import Path
from IPython.display import display
from sqlalchemy import create_engine
import datetime
import json

url = 'https://www.tideschart.com/Indonesia/Jakarta/Weekly/'

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--disable-extensions")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--no-sandbox")
user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36'
chrome_options.add_argument(f'user-agent={user_agent}')

driver = webdriver.Chrome(chrome_options = chrome_options)

time.sleep(0.5)
driver.get(url)
time.sleep(0.5)

soup = BeautifulSoup(driver.page_source, 'lxml')

#data = json.loads(soup.find("div",{"class" : "col-12 px-0"}).find("script").text)
data = json.loads(soup.find("script").text)
#data = json.loads(soup.find("script", type="application/ld+json").text)
print(data)  

C:\Users\P5CD1\AppData\Local\Temp\ipykernel_3912\1899861351.py:21: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(chrome_options = chrome_options)


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [90]:
from bs4 import BeautifulSoup
import time
from selenium import webdriver
import pandas as pd
from pathlib import Path
from IPython.display import display
from sqlalchemy import create_engine
import datetime

url = 'https://www.tideschart.com/Indonesia/Jakarta/Weekly/'

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--disable-extensions")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--no-sandbox")
user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36'
chrome_options.add_argument(f'user-agent={user_agent}')

driver = webdriver.Chrome(chrome_options = chrome_options)

time.sleep(0.5)
driver.get(url)
time.sleep(0.5)

soup = BeautifulSoup(driver.page_source, 'lxml')


df_01 = list()
for el in soup.find("div",{"class" : "card-box card-bottom pb-0"}).find_all("script"):
    #fic = text.get_text(",",strip=True)
    #print(fic)
    df_01.append(el.get_text(",{",strip=True).split(","))

df_01 = pd.DataFrame(df_01)
display(df_01)

C:\Users\P5CD1\AppData\Local\Temp\ipykernel_3912\1429088719.py:20: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(chrome_options = chrome_options)


,0,1,2,3,4,5,6,7,8,9,...,66,67,68,69,70,71,72,73,74,75
0,"const tideData = (data1 = {title: ""Daerah Khus...","unit: ""m""","\n currentDate: ""2023-01-10""","\n currentTime: ""14:14""","\n tide: [{date: ""2023-01-09""","time: ""23:36""",value: 0.3 },"{ date: ""2023-01-10""","time: ""10:49""",value: 1 },...,"sunset: ""18:15"" }","{ date: ""2023-01-15""","sunrise: ""05:49""","sunset: ""18:15"" }","{ date: ""2023-01-16""","sunrise: ""05:50""","sunset: ""18:15"" }]",\n });\n\n new VisTideChart({\n ...,\n data: tideData,\n });
1,function showHideLocations() {\n\n\t\t\t\t\t\t...,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [92]:
df_01 = df_01.drop(labels = 1, axis=0)
display(df_01)

KeyError: '[1] not found in axis'

In [93]:
df_01 = df_01.drop(df_01.columns[[0,1]], axis=1)
display(df_01)

,2,3,4,5,6,7,8,9,10,11,...,66,67,68,69,70,71,72,73,74,75
0,"\n currentDate: ""2023-01-10""","\n currentTime: ""14:14""","\n tide: [{date: ""2023-01-09""","time: ""23:36""",value: 0.3 },"{ date: ""2023-01-10""","time: ""10:49""",value: 1 },"{ date: ""2023-01-10""","time: ""22:02""",...,"sunset: ""18:15"" }","{ date: ""2023-01-15""","sunrise: ""05:49""","sunset: ""18:15"" }","{ date: ""2023-01-16""","sunrise: ""05:50""","sunset: ""18:15"" }]",\n });\n\n new VisTideChart({\n ...,\n data: tideData,\n });


In [95]:
df_01.iloc[0 , 0] = df_01.iloc[0 , 0].replace('\n', '')
display(df_01)

,2,3,4,5,6,7,8,9,10,11,...,66,67,68,69,70,71,72,73,74,75
0,"currentDate: ""2023-01-10""","\n currentTime: ""14:14""","\n tide: [{date: ""2023-01-09""","time: ""23:36""",value: 0.3 },"{ date: ""2023-01-10""","time: ""10:49""",value: 1 },"{ date: ""2023-01-10""","time: ""22:02""",...,"sunset: ""18:15"" }","{ date: ""2023-01-15""","sunrise: ""05:49""","sunset: ""18:15"" }","{ date: ""2023-01-16""","sunrise: ""05:50""","sunset: ""18:15"" }]",\n });\n\n new VisTideChart({\n ...,\n data: tideData,\n });


In [96]:
df_01.columns = range(df_01.columns.size)
display(df_01)

,0,1,2,3,4,5,6,7,8,9,...,64,65,66,67,68,69,70,71,72,73
0,"currentDate: ""2023-01-10""","\n currentTime: ""14:14""","\n tide: [{date: ""2023-01-09""","time: ""23:36""",value: 0.3 },"{ date: ""2023-01-10""","time: ""10:49""",value: 1 },"{ date: ""2023-01-10""","time: ""22:02""",...,"sunset: ""18:15"" }","{ date: ""2023-01-15""","sunrise: ""05:49""","sunset: ""18:15"" }","{ date: ""2023-01-16""","sunrise: ""05:50""","sunset: ""18:15"" }]",\n });\n\n new VisTideChart({\n ...,\n data: tideData,\n });


In [97]:
df_01.iloc[0 , 1] = df_01.iloc[0 , 1].replace('\n', '')
df_01.iloc[0 , 2] = df_01.iloc[0 , 2].replace('tide:[{', '')
display(df_01)

,0,1,2,3,4,5,6,7,8,9,...,64,65,66,67,68,69,70,71,72,73
0,"currentDate: ""2023-01-10""","currentTime: ""14:14""","\n tide: [{date: ""2023-01-09""","time: ""23:36""",value: 0.3 },"{ date: ""2023-01-10""","time: ""10:49""",value: 1 },"{ date: ""2023-01-10""","time: ""22:02""",...,"sunset: ""18:15"" }","{ date: ""2023-01-15""","sunrise: ""05:49""","sunset: ""18:15"" }","{ date: ""2023-01-16""","sunrise: ""05:50""","sunset: ""18:15"" }]",\n });\n\n new VisTideChart({\n ...,\n data: tideData,\n });


In [98]:
df_01.iloc[0 , 0] = df_01.iloc[0 , 0].replace('        ', '')
df_01.iloc[0 , 1] = df_01.iloc[0 , 1].replace('        ', '')
display(df_01)

,0,1,2,3,4,5,6,7,8,9,...,64,65,66,67,68,69,70,71,72,73
0,"currentDate: ""2023-01-10""","currentTime: ""14:14""","\n tide: [{date: ""2023-01-09""","time: ""23:36""",value: 0.3 },"{ date: ""2023-01-10""","time: ""10:49""",value: 1 },"{ date: ""2023-01-10""","time: ""22:02""",...,"sunset: ""18:15"" }","{ date: ""2023-01-15""","sunrise: ""05:49""","sunset: ""18:15"" }","{ date: ""2023-01-16""","sunrise: ""05:50""","sunset: ""18:15"" }]",\n });\n\n new VisTideChart({\n ...,\n data: tideData,\n });


In [99]:
df_01.iloc[0 , 2] = df_01.iloc[0 , 2].replace('tide: [{', '')
df_01.iloc[0 , 5] = df_01.iloc[0 , 5].replace('{ ', '')
df_01.iloc[0 , 8] = df_01.iloc[0 , 8].replace('{ ', '')
display(df_01)

,0,1,2,3,4,5,6,7,8,9,...,64,65,66,67,68,69,70,71,72,73
0,"currentDate: ""2023-01-10""","currentTime: ""14:14""","\n date: ""2023-01-09""","time: ""23:36""",value: 0.3 },"date: ""2023-01-10""","time: ""10:49""",value: 1 },"date: ""2023-01-10""","time: ""22:02""",...,"sunset: ""18:15"" }","{ date: ""2023-01-15""","sunrise: ""05:49""","sunset: ""18:15"" }","{ date: ""2023-01-16""","sunrise: ""05:50""","sunset: ""18:15"" }]",\n });\n\n new VisTideChart({\n ...,\n data: tideData,\n });


In [89]:
i = 0
while i < 71:
    df_01.iloc[:,i] = df_01.iloc[:,i].str.split(':"').str[1]
    i += 1
display(df_01)


C:\Users\P5CD1\AppData\Local\Temp\ipykernel_3912\3033691482.py:3: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df_01.iloc[:,i] = df_01.iloc[:,i].str.split(':"').str[1]


,0,1,2,3,4,5,6,7,8,9,...,64,65,66,67,68,69,70,71,72,73
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,\n });\n\n new VisTideChart({\n ...,\n data: tideData,\n });


In [102]:
df_01_temp = df_01
df_01_temp.iloc[:,0] = df_01_temp.iloc[:,0].str.split(':"').str[1]
display(df_01_temp)

AttributeError: Can only use .str accessor with string values!

In [117]:
from bs4 import BeautifulSoup
import time
from selenium import webdriver
import pandas as pd
from pathlib import Path
from IPython.display import display
from sqlalchemy import create_engine
import datetime
import re
import json

url = 'https://www.tideschart.com/Indonesia/Jakarta/Weekly/'

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--disable-extensions")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--no-sandbox")
user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36'
chrome_options.add_argument(f'user-agent={user_agent}')

driver = webdriver.Chrome(chrome_options = chrome_options)

time.sleep(0.5)
driver.get(url)
time.sleep(0.5)

soup = BeautifulSoup(driver.page_source, 'lxml')


df_01 = list()
'''for el in soup.find("div",{"class" : "card-box card-bottom pb-0"}).find("script").find_all("tide"):
    #fic = text.get_text(",",strip=True)
    #print(fic)
    df_01.append(el.get_text(",{",strip=True).split(","))

df_01 = pd.DataFrame(df_01)'''


import requests
headers =   {'user-agent':'Mozilla/5.0'}

page = requests.get(url, headers=headers)
soup = BeautifulSoup(page.content)
tes1 = soup.find("div",{"class" : "card-box card-bottom pb-0"}).find("script")
#print(soup.select_one("script:contains('title')"))

data = re.search(r"tideData = (.*});", tes1.string , flags=re.S).group(1)
data = json.loads(data)
print(data["data1 "][0]["tide"])

display(df_01)

C:\Users\P5CD1\AppData\Local\Temp\ipykernel_3912\238320255.py:22: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(chrome_options = chrome_options)


TypeError: the JSON object must be str, bytes or bytearray, not NoneType